In [14]:
import pandapower as pp
import numpy as np

In [15]:
microgrid = pp.create_empty_network(name="cub_campus")
high_voltage = 23
low_voltage = 13.8

# Xcel energy operates at 13.8kV for distribution
grid_bus1 = pp.create_bus(microgrid, name="GRID_bus1", vn_kv=high_voltage,geodata=(0,0))
#grid_bus2 = pp.create_bus(microgrid, name="GRID_bus2", vn_kv=high_voltage)
lv_bus1 = pp.create_bus(microgrid, name="LV_bus1", vn_kv=low_voltage, type="n",geodata=(0.5,0))
sb_bus1 = pp.create_bus(microgrid, name="SB_bus1", vn_kv=low_voltage, type="n",geodata=(1,0))
multidomain_bus = pp.create_bus(microgrid, name="MultiDomain_bus", vn_kv=low_voltage, type="n",geodata=(1,0.5))
central_bus = pp.create_bus(microgrid, name="CENTRAL_bus", vn_kv=low_voltage,geodata=(1.5,0))
pv_bus = pp.create_bus(microgrid, name="PV_bus", vn_kv=low_voltage,geodata=(2,0.5))
bess_bus = pp.create_bus(microgrid, name="BESS_bus", vn_kv=low_voltage, type="n",geodata=(2,1))
gen_bus = pp.create_bus(microgrid, name="Gen_bus", vn_kv=low_voltage, type="n",geodata=(0.5,0.5))
sec1 = pp.create_bus(microgrid, name="sec1", vn_kv=low_voltage, type="n",geodata=(3,1))
sec2 = pp.create_bus(microgrid, name="sec2", vn_kv=low_voltage, type="n",geodata=(3,-1))
load1 = pp.create_bus(microgrid, name="macky", vn_kv=low_voltage, type="n",geodata=(3.5,1.3))
load2 = pp.create_bus(microgrid, name="hellums", vn_kv=low_voltage, type="n",geodata=(3.5,0.7))
load3 = pp.create_bus(microgrid, name="stadium", vn_kv=low_voltage, type="n",geodata=(3,0))
load4 = pp.create_bus(microgrid, name="bookstore", vn_kv=low_voltage, type="n",geodata=(4,1))
load5 = pp.create_bus(microgrid, name="quad", vn_kv=low_voltage, type="n",geodata=(3.5,-0.7))
load6 = pp.create_bus(microgrid, name="ec", vn_kv=low_voltage, type="n",geodata=(3.5,-1.3))
load7 = pp.create_bus(microgrid, name="chw_plant", vn_kv=low_voltage, type="n",geodata=(4,-0.7))
load8 = pp.create_bus(microgrid, name="kitt", vn_kv=low_voltage, type="n",geodata=(4,-1.3))

buses = [grid_bus1, lv_bus1, sb_bus1, multidomain_bus, central_bus, pv_bus, bess_bus, sec1, sec2, load1, load2, load3, load4, load5, load6, load7, load8, gen_bus]
conns = [[1,2],[2,4],[3,4],[4,5],[4,6],[4,7],[4,11],[4,8],[7,9],[7,12],[9,10],[10,12],[8,13],[8,15],[13,14],[14,16],[16,15],[3,17]]
for conn in conns:
    pp.create_line(microgrid, buses[conn[0]], buses[conn[1]], 0.5, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV")

main_campus = pp.create_ext_grid(microgrid, grid_bus1, vm_pu=1.02, va_degree=50)
main_transformer = pp.create_transformer_from_parameters(microgrid, grid_bus1, lv_bus1, 2000, high_voltage, low_voltage, 0.25, 10, 0.48, 0.06)
main_gen = pp.create_gen(microgrid, gen_bus, p_mw=3.5)
pv_gen = pp.create_gen(microgrid, pv_bus, p_mw=0)

ps = [347.648998,189.584781,4337.210930,1923.991871,3396.980054,796.892406,972.234648,1028.719053]
ls = [load1, load2, load3, load4, load5, load6, load7, load8]
for l in range(len(ls)):
    p = ps[l] * 0.001
    s = p * np.tan(np.arccos(np.random.uniform(0.85,0.95)))
    pp.create_load(microgrid, ls[l], p_mw=p, q_mvar=s, const_i_percent=1, const_z_percent=3)
    
pp.runpp(microgrid)

In [16]:
v_b = 13800
s_b = 10**6 
z_b = (v_b**2)/s_b
microgrid.line['pu_r'] = microgrid.line['r_ohm_per_km'] / z_b
microgrid.line['pu_z'] = microgrid.line['x_ohm_per_km'] / z_b

In [17]:
import pandas as pd
x = pd.merge(microgrid.bus, microgrid.res_bus, left_index=True, right_index=True)

In [18]:
for i in ['vm_pu','va_degree']:
    x[f'data_record_{i[:2]}'] = f"parameter Real {i[:2]}_"+x['name']+"="+x[i].astype(str)+";"

In [19]:
print(x.to_string(columns=['data_record_vm'],index=False,header=False))
print(x.to_string(columns=['data_record_va'],index=False,header=False))

                    parameter Real vm_GRID_bus1=1.02;
        parameter Real vm_LV_bus1=1.0190648100614927;
         parameter Real vm_SB_bus1=1.010819370014292;
parameter Real vm_MultiDomain_bus=1.0012850847903674;
     parameter Real vm_CENTRAL_bus=1.002572878494832;
                        parameter Real vm_PV_bus=1.0;
       parameter Real vm_BESS_bus=1.0025736948913524;
        parameter Real vm_Gen_bus=0.9999999999999997;
            parameter Real vm_sec1=1.001685961237111;
           parameter Real vm_sec2=1.0006726870071982;
          parameter Real vm_macky=1.0013832665083136;
        parameter Real vm_hellums=1.0012066717131891;
        parameter Real vm_stadium=1.0009736268684934;
       parameter Real vm_bookstore=1.001099693942596;
           parameter Real vm_quad=0.9995404171689984;
              parameter Real vm_ec=0.999384459103874;
      parameter Real vm_chw_plant=0.9999052454152928;
           parameter Real vm_kitt=0.9994567822674191;
                        para

In [ ]:
x = pd.merge(microgrid.load.drop(['p_mw','q_mvar'],axis=1), microgrid.res_load, left_index=True, right_index=True)
x['name'] = (x.index+1).astype(str)

In [ ]:
for i in ['p_mw','q_mvar']:
    x[f'data_record_{i[:1]}'] = f"parameter Real {i[:1]}_load"+x['name']+"="+x[i].astype(str)+";"

In [ ]:
print(x.to_string(columns=['data_record_p'],index=False,header=False))
print(x.to_string(columns=['data_record_q'],index=False,header=False))

In [ ]:
microgrid.line['name'] = microgrid.bus.iloc[microgrid.line['to_bus']]['name'].values + "_" + microgrid.bus.iloc[microgrid.line['from_bus']]['name'].values
microgrid.line['r'] = microgrid.line['length_km'] * microgrid.line['r_ohm_per_km']
microgrid.line['x'] = microgrid.line['length_km'] * microgrid.line['x_ohm_per_km']
microgrid.line['data_record_r'] = "parameter Real " + microgrid.line['name'] + "_r=" + microgrid.line['r_ohm_per_km'].astype(str) + ";"
microgrid.line['data_record_x'] = "parameter Real " + microgrid.line['name'] + "_x=" + microgrid.line['x_ohm_per_km'].astype(str) + ";"

In [ ]:
print(microgrid.line.to_string(columns=['data_record_r'],index=False,header=False))
print(microgrid.line.to_string(columns=['data_record_x'],index=False,header=False))

In [25]:
import pandapower as pp
import math
from pandapower.pf.runpp_3ph import runpp_3ph

microgrid = pp.create_empty_network(name="cub_campus")

# Xcel energy operates at 13.8kV for distribution
grid_bus1 = pp.create_bus(microgrid, name="GRID_bus1", vn_kv=high_voltage,geodata=(0,0))
#grid_bus2 = pp.create_bus(microgrid, name="GRID_bus2", vn_kv=high_voltage)
lv_bus1 = pp.create_bus(microgrid, name="LV_bus1", vn_kv=low_voltage, type="n",geodata=(0.5,0))
sb_bus1 = pp.create_bus(microgrid, name="SB_bus1", vn_kv=low_voltage, type="n",geodata=(1,0))
multidomain_bus = pp.create_bus(microgrid, name="MultiDomain_bus", vn_kv=low_voltage, type="n",geodata=(1,0.5))
central_bus = pp.create_bus(microgrid, name="CENTRAL_bus", vn_kv=low_voltage,geodata=(1.5,0))
pv_bus = pp.create_bus(microgrid, name="PV_bus", vn_kv=low_voltage,geodata=(2,0.5))
bess_bus = pp.create_bus(microgrid, name="BESS_bus", vn_kv=low_voltage, type="n",geodata=(2,1))
gen_bus = pp.create_bus(microgrid, name="Gen_bus", vn_kv=low_voltage, type="n",geodata=(0.5,0.5))
sec1 = pp.create_bus(microgrid, name="sec1", vn_kv=low_voltage, type="n",geodata=(3,1))
sec2 = pp.create_bus(microgrid, name="sec2", vn_kv=low_voltage, type="n",geodata=(3,-1))
load1 = pp.create_bus(microgrid, name="macky", vn_kv=low_voltage, type="n",geodata=(3.5,1.3))
load2 = pp.create_bus(microgrid, name="hellums", vn_kv=low_voltage, type="n",geodata=(3.5,0.7))
load3 = pp.create_bus(microgrid, name="stadium", vn_kv=low_voltage, type="n",geodata=(3,0))
load4 = pp.create_bus(microgrid, name="bookstore", vn_kv=low_voltage, type="n",geodata=(4,1))
load5 = pp.create_bus(microgrid, name="quad", vn_kv=low_voltage, type="n",geodata=(3.5,-0.7))
load6 = pp.create_bus(microgrid, name="ec", vn_kv=low_voltage, type="n",geodata=(3.5,-1.3))
load7 = pp.create_bus(microgrid, name="chw_plant", vn_kv=low_voltage, type="n",geodata=(4,-0.7))
load8 = pp.create_bus(microgrid, name="kitt", vn_kv=low_voltage, type="n",geodata=(4,-1.3))

In [26]:
buses = [grid_bus1, lv_bus1, sb_bus1, multidomain_bus, central_bus, pv_bus, bess_bus, sec1, sec2, load1, load2, load3, load4, load5, load6, load7, load8, gen_bus]
conns = [[1,2],[2,4],[3,4],[4,5],[4,6],[4,7],[4,11],[4,8],[7,9],[7,12],[9,10],[10,12],[8,13],[8,15],[13,14],[14,16],[16,15],[3,17]]

for conn in conns:
    pp.create_line_from_parameters(microgrid, from_bus=buses[conn[0]], to_bus=buses[conn[1]], length_km=1, r_ohm_per_km=0.642, x_ohm_per_km=0.083, c_nf_per_km=210, max_i_ka=0.142, r0_ohm_per_km=1, c0_nf_per_km=1, x0_ohm_per_km=1)


In [27]:
main_campus = pp.create_ext_grid(microgrid, grid_bus1, vm_pu=1.02, va_degree=50, s_sc_max_mva=1000, rx_max=0.1, x0x_max=0.1, r0x0_max=0.1)
main_transformer = pp.create_transformer_from_parameters(microgrid, grid_bus1, lv_bus1, sn_mva=2000, vn_hv_kv=high_voltage, vn_lv_kv=low_voltage, vk_percent=6,vkr_percent=1.425,pfe_kw=1.35,i0_percent=0.3375,shift_degree=150,vector_group='Dyn',vk0_percent=1,vkr0_percent=1,mag0_percent=100,mag0_rx=0,si0_hv_partial=0.9)
main_gen = pp.create_gen(microgrid, gen_bus, p_mw=3.5)
pv_gen = pp.create_gen(microgrid, pv_bus, p_mw=0)

ps = [0.7251755596758873,0.7297487247945658,0.29527647624419634,0.276007039044313,0.28414495476850493,0.3164915177528286,0.6373714079981926,0.20932592870694763]
qs = [0.44699829145554887,0.42438198177823544,0.17202459814021925,0.14260173717495012,0.09618761821549013,0.18002382377286735,0.31316178248406723,0.06885089126511246]
ls = [load1, load2, load3, load4, load5, load6, load7, load8]
for l in range(len(ls)):
    a = 1e3
    b = 1e3*0.98
    c = 1e3*1.02
    pp. create_asymmetric_load(microgrid,bus=ls[l],p_a_mw=a*ps[l],q_a_mvar=a*qs[l],p_b_mw=b*ps[l],q_b_mvar=b*qs[l],p_c_mw=c*ps[l],q_c_mvar=c*qs[l])
    #pp.create_load(microgrid, ls[l], p_mw=ps[l], q_mvar=qs[l], const_i_percent=1, const_z_percent=3)

runpp_3ph(microgrid)

KeyError: 'res_gen_3ph'

In [28]:
pp.diagnostic(microgrid)



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------


 --------


 --------


 --------


 --------


 --------

Checking for overload...

Overload check failed: Power flow still does not converge with load and generation scaled down to 0.1 percent.

 --------


 --------

Checking switch configuration...

Power flow still does not converge with all switches closed.

 --------


 --------


 --------


 --------

numba_comparison failed due to the following error:
Power Flow nr did not converge after 10 iterations!

 --------


 --------

_____________ END OF PANDAPOWER DIAGNOSTIC _____________ 


{'overload': {'load': False, 'generation': False},
 'wrong_switch_configuration': False}

In [ ]:
# example pulled from github

net = pp.create_empty_network()
pp.add_zero_impedance_parameters(net)

bus_hv = pp.create_bus(net, vn_kv=20., name="Bus_hv ")
bus_lv = pp.create_bus(net, vn_kv=0.4, name="Bus_lv")
muff_1 = pp.create_bus(net, vn_kv=0.4, name="muff 1", type="m")

P = 0.0074
cos_phi = 0.97
Phi = math.acos(cos_phi)
Q = (math.tan(Phi)*P) / 1000000

#create bus elements
pp.create_ext_grid(net,
bus=bus_hv,
vm_pu=1.03,
name="Grid Connection",
s_sc_max_mva=1000,
rx_max=0.1,
x0x_max=0.1,
r0x0_max=0.1)

pp. create_asymmetric_load(net,
bus=muff_1,
p_a_mw=P,
q_a_mvar=Q)

pp.create_transformer_from_parameters(net,
hv_bus=bus_hv,
lv_bus=bus_lv,
name="Trafo_1",
sn_mva=0.4,
vn_hv_kv=20,
vn_lv_kv=0.4,
vk_percent=6,
vkr_percent=1.425,
pfe_kw=1.35,
i0_percent=0.3375,
shift_degree=150,
vector_group='Dyn',
vk0_percent=1,
vkr0_percent=1,
mag0_percent=100,
mag0_rx=0,
si0_hv_partial=0.9)

pp.create_line_from_parameters(net,
from_bus=bus_lv,
to_bus=muff_1,
length_km=0.01,
r_ohm_per_km=0.642,
x_ohm_per_km=0.083,
c_nf_per_km=210,
max_i_ka=0.142,
r0_ohm_per_km=1,
c0_nf_per_km=1,
x0_ohm_per_km=1)